In [3]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Bidirectional, LSTM, Dense, Input, TimeDistributed, Dropout, LayerNormalization
from sklearn.model_selection import train_test_split

In [2]:
import pandas as pd

In [4]:
df = pd.read_csv('/content/AgriDatasetAmharic - Amharic1.csv')
df.head()

,Question,Answer
0,የሙዝ ሰብል የመጣው ከየት ነው?,ደቡብ ምስራቅ እስያ
1,በደቡብ ምስራቅ እስያ የሙዝ ሰብል ምን ያህል ጊዜ ይመረታል?,በሺዎች የሚቆጠሩ ዓመታት
2,የሙዝ ሰብል በአለም አቀፍ ደረጃ ጠቃሚ የሆነው ለምንድነው?,የምግብ ዋስትና እና ኢኮኖሚያዊ ጠቀሜታ
3,ሁለቱ ዋና ዋና የሙዝ ምድቦች ምንድናቸው?,ጣፋጭ ሙዝ እና ሙዝ ማብሰል
4,ለጣፋጭ ሙዝ ሌላ ስም ማን ይባላል?,የጣፋጭ ሙዝ


In [5]:
stop_words = amharic_stop_words = set(
    ['እና', 'እንዲሁም', 'ለምሳሌ', 'በታውሾ', 'እንጂ', 'እንደ', 'ስለዚህ', 'በርሱ', 'በእነሱ', 'እነሱም', 'እርሱ',
    'ያለ', 'እንዲሁ', 'ከዚያ', 'ስለዚህ', 'ወደ', 'ከእነዚህ', 'የሆኑ', 'በታም', 'በእርግጥ', 'እንደዚህ', 'እንዲሁም', 'ምን',
    'በቀር', 'በዚህ', 'እንዲሁ', 'እንዲህ', 'ከዚህ', 'በዚሁ', 'በዚህ', 'ወይም', 'እንዲህ', 'እንዲሁም', 'ወደ', 'እነዚህ',
    'እንዲሁም', 'በርሱ', 'በእነሱ', 'እነሱም', 'እርሱ', 'ያለ', 'እንዲሁ', 'ከዚያ', 'ስለዚህ', 'ወደ', 'ከእነዚህ',
    'የሆኑ', 'በታም', 'በእርግጥ', 'እንደዚህ', 'እንዲሁም', 'ምን', 'በቀር', 'በዚህ', 'እንዲሁ', 'እንዲህ', 'ከዚህ',
    'በዚሁ', 'በዚህ', 'ወይም', 'እንዲህ', 'እንዲሁም', 'ወደ', 'እነዚህ', 'እንዲሁም', 'እና']
)


In [6]:
def remove_stop_words(text):
    print(text)
    words = text.split()
    filtered_words = [word for word in words if word not in stop_words]
    return ' '.join(filtered_words)

In [7]:
questions = []
answers = []

for question, answer in zip(df['Question'], df['Answer']):
    if type(question) == str and type(answer) == str:
      questions.append(remove_stop_words(question))
      answers.append(remove_stop_words(answer))

Streaming output truncated to the last 5000 lines.
ገበሬዎች የሸክላ አፈርን የአፈር ለምነት እንዴት ማሻሻል እንደሚችሉ.
የውሃ ማፍሰሻን አሻሽል፡ አወቃቀሩን እና የውሃ ማፍሰሻ አቅሙን ለማሻሻል እንደ ብስባሽ ወይም በደንብ የበሰበሰ ፍግ ያሉ ኦርጋኒክ ቁስ አካላትን ወደ አፈር ውስጥ ማስገባት። አፈርን አስተካክል፡- ሸካራማነቱን ለማሻሻል እና የውሃ ማፍሰሻ ችሎታውን ለመጨመር አሸዋ ወይም ሌሎች ረቂቅ ቁሶችን በሸክላ አፈር ላይ ይጨምሩ። ተገቢውን የማረስ ልምድን ተግባራዊ ማድረግ፡- መሬቱ በጣም እርጥብ በሚሆንበት ጊዜ ከመጠን በላይ ከመዝራት ወይም ከመስራት ይቆጠቡ። የታመቁ የአፈር ንጣፎችን ለመስበር እና ስር መግባቱን ለማሻሻል ተገቢውን የእርሻ ቴክኒኮችን ይጠቀሙ። ኦርጋኒክ ቁስን ይተግብሩ፡ ለምነቱን፣ አወቃቀሩን እና ውሃ የመያዝ አቅሙን ለማሳደግ በየጊዜው እንደ ብስባሽ ወይም ሽፋን ያሉ ሰብሎችን የመሳሰሉ ኦርጋኒክ ቁስ አካሎችን ይጨምሩ። የሰብል ማሽከርከርን ይለማመዱ፡-የተጨመቀ አፈርን ለመስበር እና በአፈር ወለድ ተላላፊ በሽታዎችን የመያዝ እድልን ለመቀነስ የበቆሎ ሰብሎችን የተለያየ ስርወ-ቅርጽ ካላቸው እና የንጥረ ነገር ፍላጎት ካላቸው ሰብሎች ጋር ማሽከርከር።
የሜይሊባግ ወረራዎችን ለመቆጣጠር የሚያገለግሉ ሌሎች ተባዮች አያያዝ ዘዴዎች ምንድናቸው?
ተከላካይ ዝርያዎችን መጠቀም፣ ተገቢውን የእፅዋትን አመጋገብ እና የእርጥበት መጠን መጠበቅ፣ እና በካሳቫ ተክሎች ዙሪያ የአረም እድገትን መቀነስ።
ካሳቫ ለመብሰል ምን ያህል ጊዜ ይወስዳል።
እንደ ልዩነቱ ለመብሰል ከ8 እስከ 18 ወራት ይወስዳል
አረሞችን ከመጨፍለቅ በተጨማሪ የአሌሎፓቲክ ሽፋን ሰብሎች ሌሎች ባህሪያት ምንድ ናቸው.
የበሽታ ዑደቶችን መስበር እና የባክቴሪያ እና የፈንገስ በ

In [8]:
print(questions[:10])
print(answers[:10])

['የሙዝ ሰብል የመጣው ከየት ነው?', 'በደቡብ ምስራቅ እስያ የሙዝ ሰብል ያህል ጊዜ ይመረታል?', 'የሙዝ ሰብል በአለም አቀፍ ደረጃ ጠቃሚ የሆነው ለምንድነው?', 'ሁለቱ ዋና ዋና የሙዝ ምድቦች ምንድናቸው?', 'ለጣፋጭ ሙዝ ሌላ ስም ማን ይባላል?', 'በአለም አቀፍ ንግድ በብዛትና በዋጋ ከፍተኛው ደረጃ ያለው የትኛው የሙዝ አይነት ነው?', 'የዓለም ምግብና ግብርና ድርጅት በ2012 የሙዝ ገበያ ያህል ዋጋ ነበረው?', 'በ 2012 ያህል የሙዝ ምርት ሪፖርት ተደርጓል?', 'በብዙ የአፍሪካ፣ የእስያ የላቲን አሜሪካ አገሮች ሙዝ ጠቃሚ የሆነው ለምንድነው?', 'በማደግ ላይ ባሉ አገሮች ውስጥ የሙዝ ዋና አጠቃቀም ምንድነው?']
['ደቡብ ምስራቅ እስያ', 'በሺዎች የሚቆጠሩ ዓመታት', 'የምግብ ዋስትና ኢኮኖሚያዊ ጠቀሜታ', 'ጣፋጭ ሙዝ ሙዝ ማብሰል', 'የጣፋጭ ሙዝ', 'ጣፋጭ ሙዝ', '14.01 ቢሊዮን ዶላር', '158.4 ሚሊዮን ቶን', 'የውጭ ምንዛሪ ምንጭ', 'የምግብ ዋስትና']


In [9]:
print(len(questions))
print(len(answers))

24716
24716


In [10]:
# Parameters
embedding_dim = 128
max_length = 20
trunc_type = 'post'
padding_type = 'post'
oov_token = "<OOV>"

In [11]:
# Tokenize the data
tokenizer = Tokenizer( oov_token=oov_token)
tokenizer.fit_on_texts(questions + answers)
word_index = tokenizer.word_index

# Convert texts to sequences
question_sequences = tokenizer.texts_to_sequences(questions)
answer_sequences = tokenizer.texts_to_sequences(answers)

# Pad sequences
question_padded = pad_sequences(question_sequences, maxlen=max_length, padding=padding_type, truncating=trunc_type)
answer_padded = pad_sequences(answer_sequences, maxlen=max_length, padding=padding_type, truncating=trunc_type)

# Ensure labels are correctly shaped for sparse categorical cross-entropy
answer_padded = np.expand_dims(answer_padded, -1)

In [12]:
vocab_size = len(tokenizer.word_index) +1
print(vocab_size)

15014


In [13]:
from tensorflow.keras.regularizers import l2

In [14]:
# Split the data into training and testing sets
train_questions, test_questions, train_answers, test_answers = train_test_split(question_padded, answer_padded, test_size=0.1)

# Build the model
model = Sequential([
    Embedding(vocab_size, embedding_dim, input_length=max_length),
    Bidirectional(LSTM(128, return_sequences=True, kernel_regularizer=l2(0.01))),
    Dropout(0.5),
    LayerNormalization(),
    Bidirectional(LSTM(128, return_sequences=True, kernel_regularizer=l2(0.01))),
    Dropout(0.5),
    LayerNormalization(),
    Bidirectional(LSTM(128, return_sequences=True, kernel_regularizer=l2(0.01))),
    Dropout(0.5),
    LayerNormalization(),
    TimeDistributed(Dense(vocab_size, activation='softmax', kernel_regularizer=l2(0.01)))
])

In [15]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 20, 128)           1921792   
                                                                 
 bidirectional (Bidirection  (None, 20, 256)           263168    
 al)                                                             
                                                                 
 dropout (Dropout)           (None, 20, 256)           0         
                                                                 
 layer_normalization (Layer  (None, 20, 256)           512       
 Normalization)                                                  
                                                                 
 bidirectional_1 (Bidirecti  (None, 20, 256)           394240    
 onal)                                                           
                                                        

In [16]:
# Compile the model
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Train the model
history = model.fit(train_questions, train_answers, epochs=50, validation_data=(test_questions, test_answers))

Epoch 1/50
696/696 [==============================] - 42s 44ms/step - loss: 6.1131 - accuracy: 0.4370 - val_loss: 5.1448 - val_accuracy: 0.4339
Epoch 2/50
696/696 [==============================] - 23s 33ms/step - loss: 4.9094 - accuracy: 0.4399 - val_loss: 5.0158 - val_accuracy: 0.4316
Epoch 3/50
696/696 [==============================] - 23s 33ms/step - loss: 4.7742 - accuracy: 0.4410 - val_loss: 4.8419 - val_accuracy: 0.4351
Epoch 4/50
696/696 [==============================] - 23s 33ms/step - loss: 4.6988 - accuracy: 0.4417 - val_loss: 4.7319 - val_accuracy: 0.4351
Epoch 5/50
696/696 [==============================] - 23s 33ms/step - loss: 4.6581 - accuracy: 0.4420 - val_loss: 5.0205 - val_accuracy: 0.4332
Epoch 6/50
696/696 [==============================] - 23s 33ms/step - loss: 4.6181 - accuracy: 0.4423 - val_loss: 4.5967 - val_accuracy: 0.4350
Epoch 7/50
696/696 [==============================] - 23s 33ms/step - loss: 4.5382 - accuracy: 0.4438 - val_loss: 4.5964 - val_accuracy:

In [17]:
# Evaluate the model
loss, accuracy = model.evaluate(test_questions, test_answers)
print(f"Loss: {loss}")
print(f"Accuracy: {accuracy}")


78/78 [==============================] - 1s 18ms/step - loss: 2.8036 - accuracy: 0.7229
Loss: 2.8036394119262695
Accuracy: 0.7228559851646423


In [18]:
# Function to generate answers
def generate_answer(question):
    sequence = tokenizer.texts_to_sequences([question])
    padded_sequence = pad_sequences(sequence, maxlen=max_length, padding=padding_type, truncating=trunc_type)
    prediction = model.predict(padded_sequence)
    print(prediction)
    predicted_indices = np.argmax(prediction[0], axis=-1)
    print(predicted_indices, len(predicted_indices))
    predicted_words = [tokenizer.index_word.get(index, '') for index in predicted_indices]
    return ' '.join(predicted_words).strip()

In [20]:
# Test the model
test_question = "የሙዝ ሰብል የመጣው ከየት ነው"
print(f"Question: {test_question}")
print(f"Answer: {generate_answer(test_question)}")

Question: የሙዝ ሰብል የመጣው ከየት ነው
1/1 [==============================] - 0s 24ms/step
[[[9.94341914e-04 8.35409253e-08 1.31882494e-03 ... 1.32418900e-05
   1.37491852e-05 1.35476812e-05]
  [4.56015300e-03 5.22113517e-08 3.57238413e-03 ... 8.13512816e-06
   8.58371914e-06 8.37761945e-06]
  [3.46343338e-01 2.37035049e-08 4.32585744e-04 ... 3.66323707e-06
   3.74497631e-06 3.76730031e-06]
  ...
  [1.00000000e+00 7.38268093e-17 3.35442069e-12 ... 1.16184545e-14
   1.21136774e-14 1.18304016e-14]
  [1.00000000e+00 1.07744457e-16 1.94138334e-11 ... 1.70288910e-14
   1.74281654e-14 1.73213223e-14]
  [1.00000000e+00 3.59051946e-16 6.84035883e-11 ... 5.69991035e-14
   5.74122997e-14 5.77547585e-14]]]
[557  55   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0] 20
Answer: የሚከሰተው እድገትን


In [21]:
model.save( 'model.h5')

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
